# Testing SimPy

## Tutorial 1 
https://simpy.readthedocs.io/en/latest/simpy_intro/basic_concepts.html

Using Python 3.5. **This code, and some of the text was taken straight from the tutorial! It is not mine...**

## Process

### Car

Make a car process. it requires reference to an environment so you can make new events. 
Car behavior is described using an infinite loop? So the car behavior will pass control to the simulation when the yield statment is  reached. Then, the simuation starts up from this event.

Car States
* Parking
* Driving

New states are announced by prinignt a message and the simulation time. (If you want to see simulation time yourself, use the property Envorinment.now)

Environment.timeout() tells the simulationto end.

In [2]:
def car(env):
    while True:
        print('Start parking a %d' % env.now)
        parking_duration = 5
        yield env.timeout(parking_duration)
        
        print('Start driving at %d' % env.now)
        trip_duration = 2
        yield env.timeout(trip_duration)

### Behavior
How does the car model behave?

In [3]:
import simpy
env = simpy.Environment()
env.process(car(env))
env.run(until=15)

Start parking a 0
Start driving at 5
Start parking a 7
Start driving at 12
Start parking a 14


## Process Interaction

Processes created by Environment.process() can interact. For instance, they can wait for one another, or interrupt each other.

### Waiting for a Process

We changed the car to an electric vehicle so that we can explore how processes interact. This car has the following process mmethods:

* run()
    * starts automatically when car is instatiated
* charge()
    * starts every time car parks

In [4]:
class Car(object):
    def __init__(self, env):
        self.env = env # starts the run process each time a car instance is created
        self.action = env.process(self.run())
    
    def run(self):
        while True:
            print('Start parking and charging at %d' % self.env.now)
            charge_duration = 5
            # yield process that that process() starts. wait for it to finish
            yield self.env.process(self.charge(charge_duration))
            print('Start driving at %d' % self.env.now)
            trip_duration = 2
            yield self.env.timeout(trip_duration)
    
    def charge(self,duration):
        yield self.env.timeout(duration)

Let's start the simulation!

In [5]:
import simpy
env = simpy.Environment()
car = Car(env)
env.run(until=15)

Start parking and charging at 0
Start driving at 5
Start parking and charging at 7
Start driving at 12
Start parking and charging at 14


### Interrupting A Process

Instead of waiting for the car to finish charging, interrupt the process and start driving

In [6]:
def driver(env, car):     
    yield env.timeout(3)     
    car.action.interrupt()

In [7]:
class Car2(object):
    def __init__(self,env):
        self.env = env
        self.action = env.process(self.run())
        
    def run(self):
        while True:
            print('Start parking and charging at %d' % self.env.now)
            charge_duration = 5 #might be interrupted
        
            try:
                yield self.env.process(self.charge(charge_duration))
            except simpy.Interrupt:
                # if interrupt happens, switch to driving state
                print('Was interrupted. Hope the battery is full enough...')
        
            print('Start driving at %d' % self.env.now)
            trip_duration = 2
            yield self.env.timeout(trip_duration)
    
    def charge(self, duration):
        yield self.env.timeout(duration)


Run simulation!

In [8]:
env = simpy.Environment()
car2 = Car2(env)
env.process(driver(env, car2))
env.run(until=15)

Start parking and charging at 0
Was interrupted. Hope the battery is full enough...
Start driving at 3
Start parking and charging at 5
Start driving at 10
Start parking and charging at 12


# Shared Resources

There are 3 types of possible resources in SimPy.

There is a Resource class.

## Basic Resource Usage

The car drives to a battery charging station (BCS), where there 2 charging spots. It needs to wait until one is available to use it.

In [9]:
def car3(env, name, bcs, driving_time, charge_duration):
    # Simulate driving to the BCS
    yield env.timeout(driving_time)
    
    # Request one of its charging spots
    print('%s arriving at %d' % (name, env.now))
    with bcs.request() as req:
        yield req
        
        #Charge the battery
        print('% starting to charge at %s' % (name, env.now))
        yield env.timeout(charge_duration)
        print('%s levaing the bcs at %s' % (name, env.now))

you need to wait for a resource slot until it is free. the request() method generates the event that does this. if you are using the resource slot you own it until you release it.

normally FIFO

resources reference an environment and a capacity

In [10]:
env3 = simpy.Environment()
bcs = simpy.Resource(env3, capacity=2)

In [11]:
def pass_reference(car3,env3,bcs):
    for i in range(4):
        env3.process(car3(env3, 'Car %d' %i, bcs, i*2, 5))
    return env3
pass_reference(car3, env3, bcs)

In [12]:
env3.run()

Car 0 arriving at 0
Car 0tarting to charge at 0
Car 1 arriving at 2
Car 1tarting to charge at 2
Car 2 arriving at 4
Car 0 levaing the bcs at 5
Car 2tarting to charge at 5
Car 3 arriving at 6
Car 1 levaing the bcs at 7
Car 3tarting to charge at 7
Car 2 levaing the bcs at 10
Car 3 levaing the bcs at 12


## **END OF TUTORIAL! **

## Thoughts
* faster to implement than building a simulation from scratch
* have event clock figured out, highly important
* still want to build my own simulation from scratch, but can use this to test music ideas

#### Time to move on to using SimPy to model a simulation scenario!